# 모델 체크포인트 저장 및 불러오기

## 오늘의 목표
1.체크포인트 저장의 필요성 및 개념 이해

학습 도중 최적 모델을 저장하여, 나중에 재학습하거나 평가 시점까지 이어서 사용할 수 있습니다.

학습 시간이 오래 걸리는 모델이라면, 중간에 학습을 중단하고 다시 시작할 때 유용합니다.

2.모델 상태(state_dict) 저장 및 불러오기

PyTorch에서 torch.save() 함수를 사용해 모델과 옵티마이저의 상태(state_dict)를 저장하는 방법을 배웁니다.

torch.load()와 model.load_state_dict()를 사용해 저장된 상태를 불러오는 방법을 학습합니다.

3.체크포인트 저장 전략

정기적으로(예: 에폭마다 또는 성능이 개선될 때) 저장하는 방법

Best model(최적 모델) 저장, 에폭 번호를 함께 저장하는 방법 등을 소개합니다.



### 1. 체크포인트 저장의 기본 개념
-모델 상태 (state_dict):

모델 내부의 모든 파라미터(가중치, 바이어스 등)와 버퍼(예: BatchNorm의 running_mean 등)를 저장합니다.

-옵티마이저 상태:

옵티마이저가 사용하는 gradient, 모멘텀, 학습률 등 업데이트에 필요한 상태 정보를 포함합니다.

-저장 시점:

학습 도중 매 에폭마다, 또는 Validation 성능이 개선될 때 등 원하는 시점에 저장할 수 있습니다.

In [186]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader as loader
import os
import numpy as np
import torch.nn.functional as f

In [188]:
class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.conv1=nn.Conv2d(1,32, kernel_size=3, stride=1,padding=1)
        self.conv2=nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(64*7*7,128)
        self.fc2=nn.Linear(128,10)

    def forward(self, x):
        x=f.relu(self.conv1(x))
        x=self.pool(x)
        x=f.relu(self.conv2(x))
        x=self.pool(x)
        x=x.view(-1,64*7*7)
        x=f.relu(self.fc1(x))
        x=self.fc2(x)
        return x
        

In [190]:
#학습 함수 (체크포인트 저장 포함)
def train_mode(model, train_loader,criter, optimy, num_epoch=5, save_every=2):
    model.train()
    for epoch in range(num_epoch):
        total=0.0
        for data, target in train_loader:
            optimy.zero_grad()
            out=model(data)
            loss=criter(out,target)
            loss.backward()
            optimy.step()
            total+=loss.item()
        avg_loss=total/len(train_loader)
        print(f"Epoch {epoch+1}/{num_epoch}, Training Loss: {avg_loss:.4f}")

        if (epoch+1)%save_every==0:
            save_check(model, optimy, epoch+1, filename=f"checkpoint_epoch{epoch+1}.pth")
        return model



In [192]:
#평가 함수 (eval_mode)
    model.eval()
    total = 0
    correct = 0
    all_pred = []
    all_target = []
    with torch.no_grad():
        for data, target in test_loader:
            out = model(data)
            pred = out.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += data.size(0)  # 오타: 여기서는 data.size(0)
            all_pred.extend(pred.cpu().numpy())
            all_target.extend(target.cpu().numpy())
    accuracy = 100.0 * correct / total
    return accuracy

def eval_mode(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    all_pred = []
    all_target = []
    with torch.no_grad():
        for data, target in test_loader:
            out = model(data)
            pred = out.argmax(dim=1)
            all_pred.extend(pred.cpu().numpy())
            all_target.extend(target.cpu().numpy())
            correct += pred.eq(target).sum().item()
            total += data.size(0)
    accuracy = 100.0 * correct / total
    return accuracy, np.array(all_target), np.array(all_pred)

# 체크포인트 저장 및 불러오기 함수

In [194]:
def save_check(model, optimy, epoch, filename="checkpoint.pth"):
    check={"epoch":epoch, # 현재 에폭 번호
    "model_state":model.state_dict(),# 모델 가중치
    "optimy":optimy.state_dict(),# 옵티마이저 상태(모멘텀, 학습률 등)
    "loss":loss}# 현재 손실 값 (선택 사항)
    # "scheduler_state_dict": scheduler.state_dict(),  # 사용 중이면 학습률 스케줄러 상태
    filepath=os.path.join(checkpoint_dir,filename)
    torch.save(checkpoint_dir,filename)
    print(f"Checkpoint saved at epoch {epoch}.")

In [196]:
def load_check(model, optimy, filename="checkpoint.pth"):
    checkpoint=torch.load(os.path.join(checkpoint_dir, filename))# 체크포인트 파일 로드
    model.load_state_dict(checkpoint['model_state'])# 모델 상태 복원
    optimy.load_state_state_dict(checkpoint['optimyt'])# 옵티마이저 상태 복원

    start_epoch=checkpoint['epoch']+1# 이어서 학습하려면, 현재 에폭을 복원합니다.
    print(f"Checkpoint loaded. Resuming from epoch {start_epoch}.")
    return start_epoch

In [198]:
#데이터셋 및 DataLoader 생성
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_data=datasets.MNIST(root='./data',train=True,download=True, transform=transform)
test_data=datasets.MNIST(root='./data',train=False,download=True,transform=transform)
train_loader=loader(train_data,batch_size=64,shuffle=True)
test_loader=loader(test_data,batch_size=1000,shuffle=True)

checkpoint_dir="./checkpoints"
os.makedirs(checkpoint_dir,exist_ok=True)

model=model()
print(model)
criter=nn.CrossEntropyLoss()
optimy=optim.Adam(model.parameters(),lr=0.001)

num_epoch=10
train_mode(model, train_loader,criter, optimy, num_epoch=5, save_every=2)
test_accuracy, y_true, y_pred = eval_mode(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
Epoch 1/5, Training Loss: 0.1333
Test Accuracy: 98.39%


In [202]:
# 체크포인트 저장 (모델과 옵티마이저 상태)
torch.save(model.state_dict(), 'cnn_checkpoint.pth')
torch.save(optimy.state_dict(), 'optimizer_checkpoint.pth')
print("체크포인트가 저장되었습니다.")

체크포인트가 저장되었습니다.


In [204]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

conf_mat = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)
print("Confusion Matrix:")
print(conf_mat)
print("\nClassification Report:")
print(report)


Confusion Matrix:
[[ 976    0    0    0    0    0    1    2    0    1]
 [   0 1131    1    1    0    0    0    1    1    0]
 [   1    3 1014    0    1    0    1   10    1    1]
 [   2    0    2  984    0    8    0   10    2    2]
 [   0    0    0    0  960    0    1    1    0   20]
 [   2    1    0    2    0  871    6    3    0    7]
 [   3    2    0    0    1    1  951    0    0    0]
 [   0    0    6    0    0    0    0 1018    1    3]
 [   5    2    5    0    5    0    8    3  939    7]
 [   0    2    0    1    2    2    0    5    2  995]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.98      0.98      1032
           3       1.00      0.97      0.98      1010
           4       0.99      0.98      0.98       982
           5       0.99      0.98      0.98       892
           6       0.98      0.99      0.99     